In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
options=webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--log-level=3')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(),chrome_options=options)

analyser = SentimentIntensityAnalyzer()

In [ ]:
platform_links={
    'men-tshirts':["https://www.flipkart.com/clothing-and-accessories/topwear/tshirts/pr?sid=clo,ash,ank",
                   "https://www.snapdeal.com/products/mens-tshirts-polos?sort=plrty",
                   "https://www.amazon.in/gp/bestsellers/apparel/1968120031/ref=zg_bs_pg_1?ie=UTF8&pg=1"],

    'men-shirts':["https://www.flipkart.com/clothing-and-accessories/topwear/shirt/men-shirt/casual-shirt/pr?sid=clo,ash,axc,mmk,kp7",
                  "https://www.snapdeal.com/products/casual-shirts?sort=plrty",
                  "https://www.amazon.in/gp/bestsellers/apparel/1968094031/ref=zg_bs_pg_1?ie=UTF8&pg=1"],

    'men-jeans':["https://www.flipkart.com/clothing-and-accessories/bottomwear/jeans/men-jeans/pr?sid=clo,vua,k58,i51",
                 "https://www.snapdeal.com/products/men-apparel-jeans?sort=plrty",
                 "https://www.amazon.in/gp/bestsellers/apparel/1968076031/ref=zg_bs_nav_apparel_2_1968024031"],

    'men-trousers':["https://www.flipkart.com/clothing-and-accessories/bottomwear/trouser/men-trouser/pr?sid=clo,vua,mle,lhk",
                    "https://www.snapdeal.com/products/men-apparel-trousers?sort=plrty",
                    "https://www.amazon.in/gp/bestsellers/apparel/1968125031?ref=zg_bs_pg_1&ie=UTF8&pg=1"],

    'men-floaters':["https://www.flipkart.com/mens-footwear/sandals-floaters/pr?sid=osp,cil,e83",
                   "https://www.snapdeal.com/products/mens-footwear-floater-sandals?sort=plrty",
                   "https://www.amazon.in/gp/bestsellers/shoes/1983571031/ref=zg_bs_pg_1?ie=UTF8&pg=1"],

    'men-shoes':["https://www.flipkart.com/mens-footwear/casual-shoes/pr?sid=osp,cil,e1f",
                 "https://www.snapdeal.com/products/mens-footwear-casual-shoes?sort=plrty",
                 "https://www.amazon.in/gp/bestsellers/shoes/9780814031/ref=zg_bs_pg_1?ie=UTF8&pg=1"],

    'women-dresses':["https://www.flipkart.com/womens-dresses/pr?sid=clo,odx,maj,jhy",
                     "https://www.snapdeal.com/products/dresses-gowns-jumpsuits?sort=plrty",
                     "https://www.amazon.in/gp/bestsellers/apparel/15330093031/ref=zg_bs_nav_apparel_3_11400137031"],

    'women-shirts':["https://www.flipkart.com/clothing-and-accessories/topwear/pr?sid=clo,ash&p[]=facets.ideal_for%255B%255D%3DWomen&p[]=facets.ideal_for%255B%255D%3Dwomen&otracker=categorytree&otracker=nmenu_sub_Women_0_Topwear",
                    "https://www.snapdeal.com/products/women-apparel-shirts-blouses?sort=plrty",
                    "https://www.amazon.in/gp/bestsellers/apparel/1968444031/ref=zg_bs_pg_1?ie=UTF8&pg=1"],

    'women-kurtis':["https://www.flipkart.com/womens-kurtas-kurtis/pr?sid=clo,cfv,cib,rkt",
                    "https://www.snapdeal.com/products/women-apparel-kurtis?sort=plrty",
                    "https://www.amazon.in/gp/bestsellers/apparel/1968255031/ref=zg_bs_nav_apparel_3_1968253031"],

    'women-sarees':["https://www.flipkart.com/clothing-and-accessories/saree-and-accessories/saree/women-saree/pr?sid=clo,8on,zpd,9og",
                    "https://www.snapdeal.com/products/women-apparel-sarees?sort=plrty",
                    "https://www.amazon.in/gp/bestsellers/apparel/1968256031/ref=zg_bs_nav_apparel_3_1968253031"],

    'women-skirts':["https://www.flipkart.com/clothing-and-accessories/bottomwear/skirts/women-skirts/pr?sid=clo,vua,iku,w5t",
                    "https://www.snapdeal.com/products/women-apparel-skirts?sort=plrty",
                    "https://www.amazon.in/gp/bestsellers/apparel/15330094031/ref=zg_bs_nav_apparel_3_11400137031"],

    'women-sandals':["https://www.flipkart.com/womens-footwear/~womens-sandals/pr?sid=osp,iko",
                     "https://www.snapdeal.com/products/womens-footwear-sandal?sort=plrty",
                     "https://www.amazon.in/gp/bestsellers/shoes/1983633031/ref=zg_bs_nav_shoes_3_1983578031"],

    'women-shoes':["https://www.flipkart.com/womens-footwear/~sports-casual-shoes/pr?&sid=osp,iko",
                   "https://www.snapdeal.com/products/womens-footwear-casual-shoes?sort=plrty",
                   "https://www.amazon.in/gp/bestsellers/shoes/9780815031/ref=zg_bs_nav_shoes_3_1983578031"],

    'kids-boys-ethnic':["https://www.flipkart.com/clothing-and-accessories/kurtas-ethnic-sets-and-bottoms/pr?sid=clo%2Ccfv&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DBoys&p%5B%5D=facets.ideal_for%255B%255D%3DBoys%2B%2526%2BGirls&p%5B%5D=facets.ideal_for%255B%255D%3DBaby%2BBoys%2B%2526%2BBaby%2BGirls&otracker=nmenu_sub_Baby%20%26%20Kids_0_Ethnic%20Wear",
                        "https://www.snapdeal.com/products/boys-clothing-ethnicwear?sort=plrty",
                        "https://www.amazon.in/gp/bestsellers/apparel/3659021031/ref=zg_bs_nav_apparel_2_1967851031"],

    'kids-boys-shirts':["https://www.flipkart.com/clothing-and-accessories/topwear/shirt/kids-shirt/pr?sid=clo,ash,axc,znf",
                        "https://www.snapdeal.com/products/boys-clothing-shirts?sort=plrty",
                        "https://www.amazon.in/gp/bestsellers/apparel/1967895031/ref=zg_bs_nav_apparel_2_1967851031"],

    'kids-boys-tshirts':["https://www.flipkart.com/clothing-and-accessories/topwear/tshirt/pr?sid=clo%2Cash%2Cank&p%5B%5D=facets.ideal_for%255B%255D%3DBoys&otracker=categorytree&otracker=nmenu_sub_Baby+%26+Kids_0_T-Shirts&p%5B%5D=facets.ideal_for%255B%255D%3DBaby%2BBoys",
                         "https://www.snapdeal.com/products/boys-clothing-tshirts?sort=plrty",
                         "https://www.amazon.in/gp/bestsellers/apparel/1967925031/ref=zg_bs_nav_apparel_3_1967922031"],

    'baby-boys-clothing-sets':["https://www.flipkart.com/clothing-and-accessories/kids-clothing-sets/kids-appareal-combo/pr?sid=clo%2Ceof%2C1bp&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DBaby%2BBoys&otracker=nmenu_sub_Baby+%26+Kids_0_Combos+Sets",
                               "https://www.snapdeal.com/products/boys-clothing-sets?sort=plrty",
                               "https://www.amazon.in/gp/bestsellers/apparel/1967893031/ref=zg_bs_nav_apparel_2_1967851031"],
                               
    'kids-girls-ethnic':["https://www.flipkart.com/clothing-and-accessories/ethnic-wear/pr?sid=clo%2Ccfv&p%5B%5D=facets.ideal_for%255B%255D%3DGirls&otracker=categorytree&otracker=nmenu_sub_Baby+%26+Kids_0_Ethnic+Wear&p%5B%5D=facets.ideal_for%255B%255D%3DBaby%2BGirls",
                         "https://www.snapdeal.com/products/girls-clothing-lehengas?sort=plrty",
                         "https://www.amazon.in/gp/bestsellers/apparel/3659020031/ref=zg_bs_nav_apparel_2_1967936031"],

    'kids-girls-dresses':["https://www.flipkart.com/clothing-and-accessories/dresses-and-gowns/dresses/kids-dresses/pr?sid=clo%2Codx%2Cmaj%2Czc1",
                          "https://www.snapdeal.com/products/girls-clothing-frocks?sort=plrty",
                          "https://www.amazon.in/gp/bestsellers/apparel/15966953031/ref=zg_bs_nav_apparel_2_1967936031"],
                          
    'baby-girls-clothing-sets':["https://www.flipkart.com/clothing-and-accessories/kids-clothing-sets/kids-appareal-combo/pr?sid=clo%2Ceof%2C1bp&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DBaby%2BGirls&otracker=nmenu_sub_Baby+%26+Kids_0_Combos+Sets",
                                "https://www.snapdeal.com/products/girls-clothing-sets?sort=plrty",
                                "https://www.amazon.in/gp/bestsellers/apparel/1967982031/ref=zg_bs_nav_apparel_2_1967936031"]
}
categories=list(platform_links.keys())

Amazon

In [ ]:
for category in categories:
    urls=[]
    brands=[]
    item_names=[]
    prices=[]
    stars=[]
    ratings=[]
    reviews=[]
    text_reviews=[]
    platforms=[]
    images=[]
    product_links=[]

    url=platform_links[category][2]
    if url!="":
        page_links=[url]
    else:
        page_links=[]
    
    for url in page_links:
        if len(product_links)>=40:
            break
        driver.get(url)
        for i in range(1,5):
            driver.execute_script("window.scrollTo(1,15000)")
            time.sleep(0.5)
        soup=bs(driver.page_source,"html.parser")
        links=soup.findAll("a",{"class":"a-link-normal","tabindex":"-1"})
        for link in links:
            if len(product_links)>=40:
                break
            product_links.append("https://www.amazon.in"+(link.get_attribute_list(key='href')[0]))
    
    print(len(product_links))
    
    for url in product_links:
        try:
            print(product_links.index(url))
            driver.get(url)
            brand=driver.find_element(By.XPATH,'//a[@class="a-link-normal"][@id="bylineInfo"]').text.strip()
            if "Visit" in brand:
                brand=brand[10:-6]
            else:
                brand=brand[7:]

            item_name=(driver.find_element(By.XPATH,'//span[@class="a-size-large product-title-word-break"][@id="productTitle"]').text.strip())[len(brand)+1:]
            price="₹"+driver.find_element(By.XPATH,'//span[@class="a-price-whole"]').text
            platform="Amazon"
            #image=(soup.findAll("img",{"data-a-manual-replacement":"true"})[0].get_attribute_list('src'))[0]
            image=driver.find_element(By.XPATH,'//img[@id="landingImage"]').get_attribute('src')
            urls.append(url)
            brands.append(brand)
            item_names.append(item_name)
            prices.append(price)
            platforms.append(platform)
            images.append(image)
            try:
                star=(driver.find_element(By.XPATH,'//span[@id="acrPopover"]').get_attribute('title'))[:3]
                ratrevlink=driver.find_element(By.XPATH,'//a[@data-hook="see-all-reviews-link-foot"]').get_attribute('href')
                driver.get(ratrevlink)
                rating_number=driver.find_element(By.XPATH,'//div[@data-hook="cr-filter-info-review-rating-count"]').text
                ratings_num=rating_number[0:rating_number.find(' total ratings')]
                ratings_num=int(ratings_num.replace(',',''))
                reviews_num=rating_number[rating_number.find('s,')+3:rating_number.find(' with reviews')]
                reviews_num=int(reviews_num.replace(',',''))
                stars.append(float(star))
                ratings.append(ratings_num)
                reviews.append(reviews_num)
                reviewss=[]
                x=driver.find_elements_by_xpath('//span[@data-hook="review-body"]')
                for i in range(len(x)):
                    try:
                        x[i]=x[i].find_element(By.XPATH,'./child::*')
                        reviewss.append(x[i].text)
                    except:
                        continue
                text_reviews.append(reviewss)
            except:
                stars.append(0)
                ratings.append(0)
                reviews.append(0)
                text_reviews.append([])
        except Exception as e:
            print("Incomplete Details for item: "+url)

    table={'URL':urls,'BRAND':brands,'NAME':item_names,'PRICE':prices,'STARS':stars,'NUMBER OF RATINGS':ratings,'NUMBER OF REVIEWS':reviews,'LIST OF REVIEWS':text_reviews,'PLATFORM':platforms,'IMAGE':images}
    df=pd.DataFrame(table)
    df.to_csv('ScrapedData/'+category+'-data.csv',index=False)

Flipkart

In [ ]:
for category in categories:
    urls=[]
    brands=[]
    item_names=[]
    prices=[]
    stars=[]
    ratings=[]
    reviews=[]
    text_reviews=[]
    platforms=[]
    images=[]
    product_links=[]

    url=platform_links[category][0]
    x= requests.get(url)
    soup= bs(x.content, "html.parser")
    page=(soup.findAll("a", {"class": "ge-49M"}))[:5]
    page_links=[]
    for p in page:
        page_links.append('http://flipkart.com'+p.get('href'))

    for url in page_links:
        if len(product_links)>=60:
            break
        x= requests.get(url)
        soup= bs(x.content, "html.parser")
        links=soup.findAll("a", {"class": "_2UzuFa"})
        for link in links:
            if len(product_links)>=60:
                break
            product_links.append('http://flipkart.com'+link.get('href'))

    for url in product_links: 
        try:
            print(product_links.index(url))
            x= requests.get(url)
            soup= bs(x.content, "html.parser")
            brand=soup.findAll("span", {"class": "G6XhRU"})[0].text
            item_name=soup.findAll("span", {"class": "B_NuCI"})[0].text
            price=soup.findAll("div", {"class": "_30jeq3 _16Jk6d"})[0].text
            platform="Flipkart"
            image=soup.findAll("img",{"class":"_2r_T1I _396QI4"})[0].attrs['src']

            urls.append(url)
            brands.append(brand)
            item_names.append(item_name)
            prices.append(price)
            platforms.append(platform)
            images.append(image)
            try:
                star=soup.findAll("div", {"class": "_3LWZlK _3uSWvT"})[0].text
                rating_number=soup.findAll("span", {"class": "_2_R_DZ"})[0].text 
                ratings_num=rating_number[0:rating_number.find('ratings')-1]
                ratings_num=int(ratings_num.replace(',',''))
                reviews_num=rating_number[rating_number.find('and')+4:rating_number.find('reviews')-1]
                reviews_num=int(reviews_num.replace(',',''))
                reviewss=[]
                x=soup.findAll("div", {"class": "_6K-7Co"})
                for i in range(len(x)):
                    reviewss.append(x[i].text)

                stars.append(float(star))
                ratings.append(ratings_num)
                reviews.append(reviews_num)
                text_reviews.append(reviewss)
            except:
                stars.append(0)
                ratings.append(0)
                reviews.append(0)
                text_reviews.append([])
        except:
            print("Incomplete details for item "+url)

    table={'URL':urls,'BRAND':brands,'NAME':item_names,'PRICE':prices,'STARS':stars,'NUMBER OF RATINGS':ratings,'NUMBER OF REVIEWS':reviews,'LIST OF REVIEWS':text_reviews,'PLATFORM':platforms,'IMAGE':images}
    df=pd.DataFrame(table)
    df.to_csv('ScrapedData/'+category+'-data.csv', mode='a', index=False, header=False)

Snapdeal

In [ ]:
for category in categories:
    urls=[]
    brands=[]
    item_names=[]
    prices=[]
    stars=[]
    ratings=[]
    reviews=[]
    text_reviews=[]
    platforms=[]
    images=[]
    product_links=[]

    url=platform_links[category][1]
    driver.get(url)
    for i in range(1,5):
        driver.execute_script("window.scrollTo(1,5000)")
        time.sleep(1)
    soup= bs(driver.page_source, "html.parser")
    links=soup.findAll("a",{"class":"dp-widget-link"})
    product_links=[]
    for l in links:
        if len(product_links)>=60:
            break
        link=l.get("href")
        if "product/" in link and link not in product_links:
            if len(product_links)>=60:
                break
            product_links.append(link)

    for url in product_links:
        try:
            print(product_links.index(url))
            driver.get(url)
            soup= bs(driver.page_source, "html.parser")
            brandnname=soup.findAll("h1", {"class": "pdp-e-i-head"})[0].text
            brand=brandnname[0:brandnname.find('-')-1].strip()
            item_name=brandnname[brandnname.find('-')+2:].strip()
            price=soup.findAll("span", {"class": "payBlkBig"})[0].text
            platform="Snapdeal"
            image=soup.findAll("img",{"class":"cloudzoom", "slidenum":"0"})[0].attrs['src']

            urls.append(url)
            brands.append(brand)
            item_names.append(item_name)
            prices.append(price)
            platforms.append(platform)
            images.append(image)
            try:
                star=soup.findAll("span", {"itemprop": "ratingValue"})[0].text
                ratings_num=soup.findAll("span", {"itemprop": "ratingCount"})[0].text
                reviews_num=soup.findAll("span", {"itemprop": "reviewCount"})[0].text
                stars.append(star)
                ratings.append(ratings_num)
                reviews.append(reviews_num)

                reviewss=[]
                
                #reviews not found

                if reviews_num!=0:
                    x=soup.findAll("div", {"class": "head"})
                    for r in x:
                        try:
                            reviewss.append(r.text)
                        except:
                            continue
                text_reviews.append(reviewss)
            except:
                stars.append(0)
                ratings.append(0) # type: ignore
                reviews.append(0) # type: ignore
                text_reviews.append([])

        except:
            print("Incomplete details for item"+url)

    table={'URL':urls,'BRAND':brands,'NAME':item_names,'PRICE':prices,'STARS':stars,'NUMBER OF RATINGS':ratings,'NUMBER OF REVIEWS':reviews,'LIST OF REVIEWS':text_reviews,'PLATFORM':platforms,'IMAGE':images}
    df=pd.DataFrame(table)
    df.to_csv('ScrapedData/'+category+'-data.csv', mode='a', index=False, header=False)

Data Cleaning

In [ ]:
for category in categories:
    data=pd.read_csv("ScrapedData/"+category+"-data.csv")
    data=data[['URL','BRAND','NAME','PRICE','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS','PLATFORM','IMAGE']]
    data.rename(columns = {'URL':'url','BRAND':'brand','NAME':'name','PRICE':'price','LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings','PLATFORM':'platform','IMAGE':'image'}, inplace=True)
    data.head()

    data.reviews = data.reviews.str.lower()
    data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~]','')

    data['vader_score'] = 1.0
    for ind in data.index: 
        rev=data['reviews'][ind]
        x = rev.split()
        sum_score=0.0
        for i in x:
            score = analyser.polarity_scores(i)
            sum_score=sum_score+score['compound']
        data['vader_score'][ind] = sum_score

    total_star_givers = data['num_ratings'].sum()     
    total_review_givers = data['num_reviews'].sum()

    data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

    for ind in data.index:
        score=data['final_score'][ind]
        if data['platform'][ind]=='Snapdeal':
            data['final_score'][ind]=score*50
    
    data = data.dropna()
    data = data.drop_duplicates(subset=['brand','num_reviews','platform'],keep='first')
    data = data.sort_values(by='final_score', ascending=False)
    data = data.reset_index()
    data.to_csv("FinalData/"+category+"-final.csv",index=False)